# Imports

In [1]:
import numpy as np
import sys
import pickle
import os
import pandas as pd
from itertools import compress
import scipy

# Set Directory Paths

In [2]:
# Path Dir to Git Repo
%store -r gitRepoDir
# Location of the data folder
%store -r dataDir
# Only used to load the FTDGeneralData_20221114.mat file --> Saved as NetworkDataGeneral
%store -r baseDir
# Directory path to load formatted Thickness Data
%store -r thick_dataDir
# Directory path to save Calcualted Analysis Data
%store -r thick_CalData

# [1] Loading

## Load Formatted Thickness Data
#### n x 400 (= 400 is in order 1, 2, ..., 400)

In [3]:
# HCResults
with open(os.path.join(thick_dataDir, 'HCResults.pkl'), 'rb') as f:
    HCResults = pickle.load(f)
f.close()

# PatientTAUResults
with open(os.path.join(thick_dataDir, 'PatientTAUResults.pkl'), 'rb') as f:
    PatientTAUResults = pickle.load(f)
f.close()

# PatientTDPResults
with open(os.path.join(thick_dataDir, 'PatientTDPResults.pkl'), 'rb') as f:
    PatientTDPResults = pickle.load(f)
f.close()

## Load the Look up table matching Atlas region names to Atlas Labels(Index) 

In [4]:
pathLUT = pd.read_csv(os.path.join(dataDir,'schaefer_path_20210719_20220328.csv'))

## Load the preconstructed Atlas Data

In [5]:
# loads the preconstructed Atlas data
NetworkDataGeneral = scipy.io.loadmat(os.path.join(baseDir, 'NetworkAnalysisGeneral', 'FTDGeneralData_20221114.mat'))

# [2] Define Specific Network to Use

### List of network names

In [6]:
networkNames=['DorsAttn','VentAttn','Limbic','SomMot','Default','Cont','_Vis_','All']

### Denote what NetworkNames to use

#### ii = 5 --> 'Default'
#### ii = 8 --> 'All'

In [7]:
ii = 8

### Get Names of 400 Regions in the Schaffer Atlas (in order 1, 2, ...., 400)

In [8]:
# Due to mat file issues, loading Label_Name2 from txt file / list of length 400 --> Names of 400 Regions in the Schaffer Atlas (in order)
with open(os.path.join(gitRepoDir, 'Python/LBD/NetworkDataGeneral_Schaefer400x7_LUT_Label_Name2.txt')) as file:
    Label_Name2_list = [line.rstrip() for line in file]
    file.close()

### Get List of Boolean denoting if Label_Name2_list contains our specified networkNames or not 

### Boolean list of length 400, in order of Label_Name2_List

In [9]:
if(ii == 8): # Since it's All we set all values to 1 (True) in currNetwork
    currNetwork = np.ones(len(Label_Name2_list), dtype=bool)
else: # NetworkName --> 'Default' (ii = 5)
    # -1 added because of index difference between python and matlab
    currNetwork = np.array([networkNames[ii - 1] in label_name for label_name in Label_Name2_list], dtype=bool) 

### Define saveName --> this specific case: 'Default' / 'All'

In [10]:
saveName = networkNames[ii - 1] # -1 added because of index difference between python and matlab

### Sort the pathology LUT by Label_ID400x7 1 to 400 / LUT matching 400 Atlas (thickness) regions to Pathology Region Names

In [11]:
pathLUTSorted = pathLUT.sort_values(by=['Label_ID400x7']) # Sort by label 1 to 400

### Atlas (thickness) Region Names that are in the specific network we choose (in order of the 400 regions)

In [12]:
regNamesRaw = list(compress(Label_Name2_list, currNetwork))

### Pathology Region Names that matches to regNames - Atlas Region (in order of the 400 regions)

#### Out of 400 Regions there are 91 regions that contains 'Default'

In [13]:
pathNames = pathLUTSorted[currNetwork]['PathName'].tolist()

### N = number of Thickness Regions we are analyzing for this specific network

In [14]:
N = len(regNamesRaw)

### Change the regNamesRaw (_ to -)

In [15]:
regNames = []

for j in range(N):
    nameSplit = regNamesRaw[j].replace('_', '-')
    regNames.append(nameSplit)

# Sanity Check
if ii == 5:
    assert(N == 91)
else: # ii == 8
    assert(N == 400)

# [3] Get current Center of Mass for specified current Network 

In [16]:
currCoM = (NetworkDataGeneral['NetworkDataGeneral'][0, 0]['Schaefer400x7']['CoM'][0, 0])[currNetwork]

# [4] From the Selected Thickness Network Divide into HC, TAU, and TDP

## Thickness Data

In [17]:
# Get Thickness values from HC MRI Data for only current Network [Control MRI data IDs x N] --> 54 HC x 400 regions
thickHC = HCResults['Thickness']['Mean'][:,currNetwork]
# Get Thickness values from Patient TAU MRI Data for only current Network [Control MRI data IDs x N] --> 26 TAU x 400 regions
thickTAU = PatientTAUResults['Thickness']['Mean'][:,currNetwork]
# Get Thickness values from Patient TDP MRI Data for only current Network [Control MRI data IDs x N] --> 30 TDP x 400 regions
thickTDP = PatientTDPResults['Thickness']['Mean'][:,currNetwork]

# [5] Saving Calculated Values

## Thickness Data (from selected network), HC, TAU, and TDP

In [18]:
# thickHC
with open(os.path.join(thick_CalData, 'thickHC.pkl'), 'wb') as f:
    pickle.dump(thickHC, f)
f.close()

# thickTAU
with open(os.path.join(thick_CalData, 'thickTAU.pkl'), 'wb') as f:
    pickle.dump(thickTAU, f)
f.close()

# thickTDP
with open(os.path.join(thick_CalData, 'thickTDP.pkl'), 'wb') as f:
    pickle.dump(thickTDP, f)
f.close()

## N = number of Thickness Regions we are analyzing for this specific network

In [19]:
# N
with open(os.path.join(thick_CalData, 'N.pkl'), 'wb') as f:
    pickle.dump(N, f)
f.close()

## regNames --> Atlas (thickness) Region Names that are in the specific network we choose (in order of the 400 regions)

In [20]:
# regNames
with open(os.path.join(thick_CalData, 'regNames.pkl'), 'wb') as f:
    pickle.dump(regNames, f)
f.close()

## currCoM --> Center of Mass of the Selected Network

In [21]:
# currCoM
with open(os.path.join(thick_CalData, 'currCoM.pkl'), 'wb') as f:
    pickle.dump(currCoM, f)
f.close()